In [1]:
import pandas as pd
from transformers import pipeline


2025-05-01 15:57:35.717789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746115055.952365      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746115056.020040      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Загрузка данных
df = pd.read_csv('/kaggle/input/mephikack2/cleared.csv')


In [5]:
def concat_text(row):
    
    text = str(row['doc_text'])
    
    if not pd.isna(str(row['image2text'])):
        text += str(row['image2text'])

    if not pd.isna(str(row['speech2text'])):
        text += str(row['speech2text'])

    return text.lower()

df['text'] = df.apply(concat_text, axis=1)

In [6]:
df = df.drop(columns=['doc_text', 'image2text', 'speech2text'])

In [7]:
df.describe()

,темы,text
count,5700,5700
unique,40,5320
top,['спорт'],камеди вуменnannan
freq,1305,10


In [8]:
df['class'] = df['темы']
df = df.drop(columns=['темы'])

In [9]:


# Инициализация классификатора
classifier = pipeline("zero-shot-classification", 
                     model="facebook/bart-large-mnli",
                     device=0)  # Использование GPU если доступно

# Список категорий
categories = ['личная жизнь', 'политика', 'реклама', 'соцсети', 'спорт', 'юмор']

# Функция для классификации
def classify_post(text):
    result = classifier(text, 
                       candidate_labels=categories,
                       multi_label=True)
    return {label: 1 if label in result['labels'][:2] else 0 for label in categories}

# Применение к данным
results = df['text'].apply(lambda x: classify_post(x))
df_res = df.join(pd.DataFrame(results.tolist()))

df_res.head()


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


KeyboardInterrupt: 